In [ ]:
import time
import datetime
import requests
import pandas as pd
import matplotlib.pyplot as plt

In [14]:
def GetHistoricalData(stock, start, end, frequency):
    
    import time
    import datetime
    import requests
    import pandas as pd
    
    # Set Up
    start = int(time.mktime(time.strptime(start, "%m/%d/%Y")))
    end = int(time.mktime(time.strptime(end, "%m/%d/%Y")))
    assert frequency in ['daily', 'weekly', 'monthly'], 'Frequency has to be daily, weekly, or monthly'
    if frequency == 'daily':
        frequency = '1d'
    elif frequency == 'weekly':
        frequency = '1wk'
    else:
        frequency = '1m'
    
    # Get Data
    YahooFinance = 'https://query1.finance.yahoo.com/v7/finance/download/{stock}?period1={start}&period2={end}&interval={frequency}&events=history&crumb=Vp5JJJIY4i/'
    headers = {
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36',
        'cookie': 'B=a2qoc01dom2ob&b=3&s=hv; GUC=AQEBAQFbyD5cnEIe4gR7&s=AQAAAKXb1OzH&g=W8bwqg; PRF=t%3DNFLX%252BSPY%252BRTY%253DF%252BRUTH%252B%255EIXIC%26fin-trd-cue%3D1',
        'pragma': 'no-cache',
        'referer': 'https://finance.yahoo.com/quote/{stock}/history?period1={start}&period2={end}&interval={frequency}&filter=history&frequency={frequency}'.format(stock=stock, frequency = frequency, start=start, end=end),
        'upgrade-insecure-requests': '1'
    }

    url = YahooFinance.format(stock=stock, frequency = frequency, start=start, end=end)
    response = requests.get(url, headers=headers)
    text = response.text
    lines =  text.split('\n')

    # Save Data
    title = lines[0]
    Date = []
    Open = []
    High = []
    Low = []
    Close = []
    Adj_Close = []
    Volume = []

    for line in lines[1:-1]:
        [d,o,h,l,c,ac,v] = line.split(',')
        Date.append(d)
        Open.append(float(o))
        High.append(float(h))
        Low.append(float(l))
        Close.append(float(c))
        Adj_Close.append(float(ac))
        Volume.append(float(v))
    
    # Convert to DataFrame
    df = pd.DataFrame({'Date': Date,
                        'Open': Open,
                        'High': High,
                        'Low': Low,
                        'Close': Close,
                        'Adj_Close': Adj_Close,
                        'Volume': Volume
                        }, columns=['Date','Open','High','Low','Close','Adj_Close','Volume'])
    df.index = pd.to_datetime(df.Date)
    df.drop("Date", inplace=True, axis=1)
    df = df.sort_index()
    
    return df


In [15]:
stock = 'GOOG'
start = '01/01/2005'
end = '2/19/2019'
frequency = 'daily'
df = GetHistoricalData(stock, start, end, frequency)
df.head()

,Open,High,Low,Close,Adj_Close,Volume
Date,,,,,,
2005-01-03,98.062202,101.162041,97.098465,100.700043,100.700043,31894300.0
2005-01-04,100.049278,100.809334,96.114868,96.621567,96.621567,27690700.0
2005-01-05,96.099960,97.813812,95.493904,96.129768,96.129768,16580200.0
2005-01-06,96.909698,97.317047,93.253479,93.665794,93.665794,20909100.0
2005-01-07,94.704041,96.497375,93.780052,96.298668,96.298668,19451500.0
